# Exploring the Impact of Structured Representations in Scenario Generation Based on Large Language Models

## Preparation

In [ ]:
import os
os.chdir('/content')

# fetch codebase
CODE_DIR = 'lctgen'
os.makedirs(f'./{CODE_DIR}', exist_ok=True)
!git clone https://github.com/Ariostgx/lctgen.git $CODE_DIR
os.chdir('/content/lctgen')

In [ ]:
%pip install -q -r requirements_colab.txt --quiet
%pip install ipympl --quiet

In [ ]:
!gdown https://drive.google.com/uc?id=17_TI-q4qkCOt988spWIZCqDLkZpMSptO -O data.zip
!unzip data.zip -d /content/lctgen/data/demo/waymo/

In [ ]:
!gdown https://drive.google.com/uc?id=1_s_35QO6OiHHgDxHHAa7Djadm-_I7Usr -O example.ckpt
!mkdir /content/lctgen/checkpoints
!mv example.ckpt /content/lctgen/checkpoints

In [ ]:
from google.colab import output

output.enable_custom_widget_manager()

## Setup

In [ ]:
import torch

import itertools
import json
import math
import sys

import ipywidgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

from datetime import datetime
from IPython.display import HTML, Javascript
from matplotlib.patches import Wedge
from shapely.geometry import Point, Polygon, LineString

# LCTGen
from lctgen.config.default import get_config
from lctgen.core.registry import registry
from lctgen.datasets.waymo_open_motion import WaymoOpenMotionDataset
from lctgen.inference.utils import output_formating_cot, map_retrival, get_map_data_batch, load_all_map_vectors
from lctgen.models.utils import visualize_input_seq, visualize_output_seq, transform_traj_output_to_waymo_agent

# TrafficGen
from trafficgen.utils.typedef import *
from trafficgen.utils.data_process.agent_process import WaymoAgent

In [ ]:
in_colab = "google.colab" in sys.modules

In [ ]:
cfg_file = 'cfgs/demo_inference.yaml'
cfg = get_config(cfg_file)

model_cls = registry.get_model(cfg.MODEL.TYPE)
model = model_cls.load_from_checkpoint(cfg.LOAD_CHECKPOINT_PATH, config=cfg, metrics=[], strict=False)
model.eval()

map_data_file = 'data/demo/waymo/demo_map_vec.npy'
map_vecs, map_ids = load_all_map_vectors(map_data_file)